In [1]:
%run ./work/lib.py

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [3]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/12/17 22:55:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/17 22:55:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
TABLE_NAME = 'fact_order'

In [5]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/orders_dataset.csv'.format(LAKE_HOME), header=True)

df_order = \
    df.select(
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('customer_id').cast(StringType()).alias('ID_CUSTOMER'),
        col('order_status').cast(StringType()).alias('ORD_STATUS'),
        col('order_purchase_timestamp').cast(TimestampType()).alias('DAT_PURCHASE'),
        col('order_approved_at').cast(TimestampType()).alias('DAT_APPROVED'),
        col('order_delivered_carrier_date').cast(TimestampType()).alias('DAT_SHIPPED'),
        col('order_delivered_customer_date').cast(TimestampType()).alias('DAT_DELIVERED'),
        col('order_estimated_delivery_date').cast(TimestampType()).alias('DAT_ESTIMATED_DELIVERY')
    )

customer_sk = map_sk(spark=spark, table_name='dim_customer')

In [6]:
sk = create_sk(spark=spark, df=df_order, key_column_name='ID_ORDER', table_name=TABLE_NAME)

fact_order = \
    df_order \
        .withColumn('SK_ORDER', locate_sk(sk)(col('ID_ORDER'))) \
        .withColumn('SK_CUSTOMER', locate_sk(customer_sk)(col('ID_CUSTOMER')))

In [7]:
fact_order = \
    fact_order.select(
        col('SK_ORDER'),
        col('SK_CUSTOMER'),
        col('ID_ORDER'),
        col('ORD_STATUS'),
        col('DAT_PURCHASE'),
        col('DAT_APPROVED'),
        col('DAT_SHIPPED'),
        col('DAT_DELIVERED'),
        col('DAT_ESTIMATED_DELIVERY'),   
    )

fact_order.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [8]:
spark.stop()